In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [4]:
ls

depression_submissions.zst             reddit_data_preprocessing_happy.ipynb
happy_submissions.zst                  reddit_data_preprocessing_v1.ipynb


In [ ]:
## step1. load downloaded submission reddit dump 

In [2]:
unexpectedlyWholesome_submissions =\
    pd.read_json('UnexpectedlyWholesome_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 2000000)

In [3]:
# convert the timestamp to utc
unexpectedlyWholesome_submissions['created_utc'] =\
    unexpectedlyWholesome_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [41]:
## step2. download the file into parquet file to speed up data load later stage

In [4]:
unexpectedlyWholesome_submissions.to_parquet('unexpectedlyWholesome_submissions.parquet', engine='pyarrow')

In [43]:
## step3. read as parquet file 

In [5]:
unexpectedlyWholesome_submissions = pd.read_parquet('unexpectedlyWholesome_submissions.parquet',engine='pyarrow')
unexpectedlyWholesome_submissions

,num_comments,id,secure_media_embed,title,author,domain,secure_media,retrieved_on,quarantine,hidden,stickied,distinguished,thumbnail,created_utc,spoiler,locked,link_flair_css_class,gilded,over_18,media_embed,brand_safe,is_self,link_flair_text,edited,author_flair_text,hide_score,suggested_sort,score,author_flair_css_class,media,subreddit,url,subreddit_id,contest_mode,permalink,selftext,archived,preview,post_hint,view_count,thumbnail_height,thumbnail_width,can_gild,is_video,approved_at_utc,banned_at_utc,can_mod_post,is_crosspostable,num_crossposts,parent_whitelist_status,whitelist_status,is_reddit_media_domain,pinned,subreddit_type,crosspost_parent,crosspost_parent_list,mod_note,mod_reason_by,mod_reason_title,no_follow,send_replies,subreddit_subscribers,author_cakeday,approved_by,author_flair_background_color,author_flair_template_id,author_flair_text_color,banned_by,category,clicked,content_categories,created,downs,is_original_content,likes,link_flair_richtext,link_flair_text_color,link_flair_type,media_only,mod_reports,name,num_reports,post_categories,pwls,removal_reason,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,visited,wls,author_flair_richtext,author_flair_type,author_created_utc,author_fullname,is_meta,link_flair_background_color,link_flair_template_id,subreddit_name_prefixed,media_metadata,author_patreon_flair,gildings,is_robot_indexable,all_awardings,total_awards_received,allow_live_comments,discussion_type,author_premium,awarders,removed_by,removed_by_category,treatment_tags,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,poll_data,gallery_data,is_gallery,call_to_action
0,0,5vyq4k,"{'content': None, 'height': None, 'media_domai...","Wholesome TIFU, the wholesomeness is spreading...",[deleted],i.reddituploads.com,None,1.489584e+09,0.0,False,False,None,default,2017-02-24 17:19:00,False,False,None,0,False,"{'content': None, 'height': None, 'media_domai...",0.0,False,None,0,None,0.0,NaN,3,None,None,UnexpectedlyWholesome,https://i.reddituploads.com/2838717d226440eca7...,t5_3j5py,False,/r/UnexpectedlyWholesome/comments/5vyq4k/whole...,[deleted],0.0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,NaN,None,None,None,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
1,0,5vzvvf,"{'content': None, 'height': None, 'media_domai...","Wholesome TIFU, the wholesomeness is spreading...",Shylo132,i.reddituploads.com,None,1.489585e+09,0.0,False,False,None,https://b.thumbs.redditmedia.com/Krq6OZswxOMid...,2017-02-24 20:28:11,False,False,None,0,False,"{'content': None, 'height': None, 'media_domai...",0.0,False,None,0,None,0.0,NaN,21,None,None,UnexpectedlyWholesome,https://i.reddituploads.com/2838717d226440eca7...,t5_3j5py,False,/r/UnexpectedlyWholesome/comments/5vzvvf/whole...,,0.0,"{'enabled': True, 'images': [{'id': 'jeRd9ZUs_...",link,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,NaN,None,None,None,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None,None,None,NaN,None
2,3,5w0c0e,"{'content': None, 'height': None, 'media_domai...",When the mod toolbox decides to compliment you...,I_Am_Batgirl,i.redd.it,None,1.489585e+09,0.0,False,False,None,https://b.thumbs.redditmedia.com/Y0F3SXwiGRAh1...,2017-02-24 21:44:53,False,False,None,0,False,"{'content': None, 'height': None, 'media_domai...",0.0,False,None,0,None,0.0,NaN,33,None,None,UnexpectedlyWholesome,https://i.redd.it/su87wyq2lvhy.jpg,t5_3j5py,False,/r/UnexpectedlyWholesome/comments/5w0c0e/when_...,,0.0,"{'enabled': True, 'images': [{'id': 'kI9O

In [45]:
## step4. feature selection : select only necessary columns

In [6]:
unexpectedlyWholesome_submissions =\
    unexpectedlyWholesome_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
unexpectedlyWholesome_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,UnexpectedlyWholesome,[deleted],False,None,"Wholesome TIFU, the wholesomeness is spreading...",[deleted],https://i.reddituploads.com/2838717d226440eca7...,2017-02-24 17:19:00
1,UnexpectedlyWholesome,Shylo132,False,None,"Wholesome TIFU, the wholesomeness is spreading...",,https://i.reddituploads.com/2838717d226440eca7...,2017-02-24 20:28:11
2,UnexpectedlyWholesome,I_Am_Batgirl,False,None,When the mod toolbox decides to compliment you...,,https://i.redd.it/su87wyq2lvhy.jpg,2017-02-24 21:44:53
3,UnexpectedlyWholesome,ThatAstronautGuy,False,None,Nothing like the restaurant helping you with y...,,http://i.imgur.com/tHVYkE9.jpg,2017-02-26 18:58:47
4,UnexpectedlyWholesome,[deleted],False,None,Unexpectedly wholesome and really thoughtful,[deleted],https://i.redd.it/uou0o6byz7jy.jpg,2017-03-03 16:32:46
...,...,...,...,...,...,...,...,...
10919,UnexpectedlyWholesome,[deleted],False,None,Awwh,[deleted],,2022-12-31 14:02:19
10920,UnexpectedlyWholesome,[deleted],False,None,Never truly apart,[deleted],,2022-12-31 14:10:55
10921,UnexpectedlyWholesome,letsstartanew2,False,None,"2022 is finally over! Come 2023, you beautiful!",,https://youtu.be/mpJ76rY1Krs,2022-12-31 14:30:46
10922,UnexpectedlyWholesome,a2h1sh,True,None,The boys,,https://v.redd.it/6vjnus0uq59a1,2022-12-31 15:32:22


In [48]:
## step5. basic pre-processing 

In [7]:
# remove blank body posts
unexpectedlyWholesome_submissions = \
    unexpectedlyWholesome_submissions[unexpectedlyWholesome_submissions['selftext'].apply(lambda x : len(x)!=0)]
unexpectedlyWholesome_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,UnexpectedlyWholesome,[deleted],False,None,"Wholesome TIFU, the wholesomeness is spreading...",[deleted],https://i.reddituploads.com/2838717d226440eca7...,2017-02-24 17:19:00
4,UnexpectedlyWholesome,[deleted],False,None,Unexpectedly wholesome and really thoughtful,[deleted],https://i.redd.it/uou0o6byz7jy.jpg,2017-03-03 16:32:46
5,UnexpectedlyWholesome,[deleted],False,None,The nicest person in my life is my phone,[deleted],https://imgur.com/a/SafTC,2017-03-14 18:33:41
8,UnexpectedlyWholesome,[deleted],False,None,Unexpected wholesome bot told me to post this ...,[deleted],https://i.redd.it/gkw87v3xz1py.jpg,2017-04-02 02:47:48
11,UnexpectedlyWholesome,[deleted],False,None,Not a meme but it made my heart full :),[deleted],https://i.reddituploads.com/c936928cf3844d0f8e...,2017-04-09 16:16:08
...,...,...,...,...,...,...,...,...
10908,UnexpectedlyWholesome,lavashower405,False,None,my kitten climbed onto a mattress and fell asleep,his name is ozzy.,https://www.reddit.com/gallery/zyom1b,2022-12-30 02:38:42
10910,UnexpectedlyWholesome,EmuInternational7686,False,Removed: Posts Should Be Unexpectedly Wholesome,Finding your family alive at your war zone home,[removed],https://v.redd.it/ioyo0ostv09a1,2022-12-30 11:42:14
10911,UnexpectedlyWholesome,[deleted],False,None,night shift romance,[deleted],,2022-12-30 14:12:03
10919,UnexpectedlyWholesome,[deleted],False,None,Awwh,[deleted],,2022-12-31 14:02:19


In [8]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
unexpectedlyWholesome_submissions[unexpectedlyWholesome_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,UnexpectedlyWholesome,[deleted],False,None,"Wholesome TIFU, the wholesomeness is spreading...",[deleted],https://i.reddituploads.com/2838717d226440eca7...,2017-02-24 17:19:00
4,UnexpectedlyWholesome,[deleted],False,None,Unexpectedly wholesome and really thoughtful,[deleted],https://i.redd.it/uou0o6byz7jy.jpg,2017-03-03 16:32:46
5,UnexpectedlyWholesome,[deleted],False,None,The nicest person in my life is my phone,[deleted],https://imgur.com/a/SafTC,2017-03-14 18:33:41
8,UnexpectedlyWholesome,[deleted],False,None,Unexpected wholesome bot told me to post this ...,[deleted],https://i.redd.it/gkw87v3xz1py.jpg,2017-04-02 02:47:48
11,UnexpectedlyWholesome,[deleted],False,None,Not a meme but it made my heart full :),[deleted],https://i.reddituploads.com/c936928cf3844d0f8e...,2017-04-09 16:16:08
...,...,...,...,...,...,...,...,...
10906,UnexpectedlyWholesome,[deleted],False,None,Daddy was hit by a drunk driver but doesn't le...,[deleted],,2022-12-29 12:22:58
10910,UnexpectedlyWholesome,EmuInternational7686,False,Removed: Posts Should Be Unexpectedly Wholesome,Finding your family alive at your war zone home,[removed],https://v.redd.it/ioyo0ostv09a1,2022-12-30 11:42:14
10911,UnexpectedlyWholesome,[deleted],False,None,night shift romance,[deleted],,2022-12-30 14:12:03
10919,UnexpectedlyWholesome,[deleted],False,None,Awwh,[deleted],,2022-12-31 14:02:19


In [9]:
unexpectedlyWholesome_submissions =\
    unexpectedlyWholesome_submissions[~unexpectedlyWholesome_submissions['selftext'].isin(opt_out_list)]
print(len(unexpectedlyWholesome_submissions))
unexpectedlyWholesome_submissions

391


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
30,UnexpectedlyWholesome,I_Am_Batgirl,False,None,Know what's unexpectedly wholesome? A wholesom...,As you know r/UnexpectedlyWholesome is part of...,https://www.reddit.com/r/UnexpectedlyWholesome...,2017-06-05 19:17:54
58,UnexpectedlyWholesome,LaTraLaTrill,False,None,Run the jewels at lollapolooza,Run the Jewels is currently live at lollapoloo...,https://www.reddit.com/r/UnexpectedlyWholesome...,2017-08-05 01:54:33
98,UnexpectedlyWholesome,I_Am_Batgirl,False,None,Join us as we Partner with Team Wholesome Meme...,We need your help! We’re gearing up our effort...,https://www.reddit.com/r/UnexpectedlyWholesome...,2017-09-26 22:58:40
163,UnexpectedlyWholesome,letflowgo,False,None,Sometimes a great thank you comes unexpectedly...,I had just told a lady “thank you” at Starbuck...,https://www.reddit.com/r/UnexpectedlyWholesome...,2017-12-07 04:15:04
411,UnexpectedlyWholesome,seth5124,False,None,Tried to dad joke a chatbot. It's too busy bei...,https://i.redd.it/1wkwhivgbpb11.png,https://www.reddit.com/r/UnexpectedlyWholesome...,2018-07-23 13:48:56
413,UnexpectedlyWholesome,otterykhaleesi,False,None,"Hit on me? Nope, just recommend me podcasts.",Okay so a quick scroll through this sub shows ...,https://www.reddit.com/r/UnexpectedlyWholesome...,2018-07-24 14:55:17
479,UnexpectedlyWholesome,Alphaajr,False,None,today i taught a 10 year old a valuable lesson...,So i was browsing reddit and stumbled upon a p...,https://www.reddit.com/r/UnexpectedlyWholesome...,2018-09-12 21:46:33
491,UnexpectedlyWholesome,HECUBoi,False,None,Discord Story,So I'm in this discord role-play server(It's S...,https://www.reddit.com/r/UnexpectedlyWholesome...,2018-09-21 06:10:58
499,UnexpectedlyWholesome,ClearCounter,False,None,Strangely Wholesome Amazon Customer Service Tr...,"&gt;12:27 PM PDT **Ashraf(Amazon)**: Hello, my...",https://www.reddit.com/r/UnexpectedlyWholesome...,2018-09-27 19:45:12
503,UnexpectedlyWholesome,lamy0720,False,None,Someone posted to another subreddit called “hi...,"Hippity hippity, hoppity hoppity / My tail is ...",https://www.reddit.com/r/UnexpectedlyWholesome...,2018-09-29 13:28:36


In [51]:
# step6. final data quality check 

In [12]:
print(f'Author : {unexpectedlyWholesome_submissions.iloc[[3],:].author.tolist()[0]}')
print(f'Subreddit : {unexpectedlyWholesome_submissions.iloc[[3],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {unexpectedlyWholesome_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {unexpectedlyWholesome_submissions.iloc[[3],:].created_utc.tolist()[0]}')
print()
print(f'Title : {unexpectedlyWholesome_submissions.iloc[[3],:].title.tolist()[0]}')
print(f'Body : {unexpectedlyWholesome_submissions.iloc[[3],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {unexpectedlyWholesome_submissions.iloc[[5],:].author.tolist()[0]}')
print(f'Subreddit : {unexpectedlyWholesome_submissions.iloc[[5],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {unexpectedlyWholesome_submissions.iloc[[5],:].over_18.tolist()[0]}')
print(f'Posted : {unexpectedlyWholesome_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {unexpectedlyWholesome_submissions.iloc[[5],:].title.tolist()[0]}')
print(f'Body : {unexpectedlyWholesome_submissions.iloc[[5],:].selftext.tolist()[0]}')


Author : letflowgo
Subreddit : UnexpectedlyWholesome
Teenager(True/False) : False
Posted : 2017-12-07 04:15:04

Title : Sometimes a great thank you comes unexpectedly to brighten your day
Body : I had just told a lady “thank you” at Starbucks when i picked up my drink when she told me “anytime, any day, you have a beautiful day.” Warms my heart when people go out of their way to make others feel welcome. 



Author : otterykhaleesi
Subreddit : UnexpectedlyWholesome
Teenager(True/False) : False
Posted : 2017-08-05 01:54:33

Title : Hit on me? Nope, just recommend me podcasts.
Body : Okay so a quick scroll through this sub shows me it’s mostly images but I just wanted somewhere to share this.

I was having lunch in a fast food chain today because fuck it, treat yoself! And I was wearing a shirt with the Jaws cover art on it (important I promise).

As I sat down, a guy opposite me got my attention and started just gushing about how much he liked Jaws; the book, the movie. It was amazing!
